In [1]:
### PROJET 3 : RÉALISEZ UNE ÉTUDE DE SANTÉ PUBLIQUE - Code des réponses

In [2]:
# Importation des librairies :
import numpy as np
import pandas as pd

In [3]:
# Importation des fichiers
fr_animaux = pd.read_csv("fr_animaux.csv")
fr_cereales = pd.read_csv("fr_cereales.csv")
fr_population = pd.read_csv("fr_population.csv")
fr_sousalimentation = pd.read_csv("fr_sousalimentation.csv",decimal=".",delimiter=",")
fr_vegetaux = pd.read_csv("fr_vegetaux.csv")

In [4]:
# QUESTION 1 -------------------------------------------------------------------------------------------------------------------
# Calculer le nombre total d'humains sur la planète.

In [5]:
# Aperçu rapide du tableau "fr_population"(tête) :
fr_population.head()

,Code Domaine,Domaine,Code zone,Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole
0,FBSH,Bilans Alimentaire (Ancienne méthodologie et p...,2,Afghanistan,511,Population totale,2501,Population,2013,2013,1000 personnes,30552,NaN,Donnée officielle
1,FBSH,Bilans Alimentaire (Ancienne méthodologie et p...,202,Afrique du Sud,511,Population totale,2501,Population,2013,2013,1000 personnes,52776,NaN,Donnée officielle
2,FBSH,Bilans Alimentaire (Ancienne méthodologie et p...,3,Albanie,511,Population totale,2501,Population,2013,2013,1000 personnes,3173,NaN,Donnée officielle
3,FBSH,Bilans Alimentaire (Ancienne méthodologie et p...,4,Algérie,511,Population totale,2501,Population,2013,2013,1000 personnes,39208,NaN,Donnée officielle
4,FBSH,Bilans Alimentaire (Ancienne méthodologie et p...,79,Allemagne,511,Population totale,2501,Population,2013,2013,1000 personnes,82727,NaN,Donnée officielle


In [6]:
# Suppression des entrées inutiles pour un nouveau dataframe nommé "Population" :
population = fr_population.drop(columns=["Code Domaine","Domaine","Code zone","Code Élément","Élément","Code Produit","Produit","Code année","Symbole","Description du Symbole","Année","Unité"])

population.head()

,Zone,Valeur
0,Afghanistan,30552
1,Afrique du Sud,52776
2,Albanie,3173
3,Algérie,39208
4,Allemagne,82727


In [7]:
# Calcul de la population mondiale :
population["Valeur"].sum()*1000

8413993000

In [8]:
# La population mondiale calculée en fonction des données "fr_population.csv" indique :
# 8,414 milliards d'individus !!! 
# Supposition : faute de frappe ; la valeur d'un pays a peut-être un chiffre de trop.

# Vérification :
population.sort_values(by = "Valeur")

,Zone,Valeur
143,Saint-Kitts-et-Nevis,54
20,Bermudes,65
47,Dominique,72
6,Antigua-et-Barbuda,90
89,Kiribati,102
...,...,...
76,Indonésie,249866
55,États-Unis d'Amérique,320051
75,Inde,1252140
36,"Chine, continentale",1385567


In [9]:
# Les chiffres de la population chinoise continentale semblent inclus dans ceux de la population chinoise totale.
# Vérification visuelle :
population.Zone.unique()

array(['Afghanistan', 'Afrique du Sud', 'Albanie', 'Algérie', 'Allemagne',
       'Angola', 'Antigua-et-Barbuda', 'Arabie saoudite', 'Argentine',
       'Arménie', 'Australie', 'Autriche', 'Azerbaïdjan', 'Bahamas',
       'Bangladesh', 'Barbade', 'Bélarus', 'Belgique', 'Belize', 'Bénin',
       'Bermudes', 'Bolivie (État plurinational de)',
       'Bosnie-Herzégovine', 'Botswana', 'Brésil', 'Brunéi Darussalam',
       'Bulgarie', 'Burkina Faso', 'Cabo Verde', 'Cambodge', 'Cameroun',
       'Canada', 'Chili', 'Chine', 'Chine - RAS de Hong-Kong',
       'Chine - RAS de Macao', 'Chine, continentale',
       'Chine, Taiwan Province de', 'Chypre', 'Colombie', 'Congo',
       'Costa Rica', "Côte d'Ivoire", 'Croatie', 'Cuba', 'Danemark',
       'Djibouti', 'Dominique', 'Égypte', 'El Salvador',
       'Émirats arabes unis', 'Équateur', 'Espagne', 'Estonie',
       'Eswatini', "États-Unis d'Amérique", 'Éthiopie',
       'Fédération de Russie', 'Fidji', 'Finlande', 'France', 'Gabon',
       'Gam

In [10]:
# L'occurrence "Chine" est présente à cinq reprises : 

# "Chine"
# "Chine - RAS de Hong-Kong"
# "Chine - RAS de Macao"
# "Chine, continentale"
# "Chine, Taiwan Province de"

# Après vérification, l'occurrence "Chine" regroupe déjà en son sein les populations de Hong-Kong,
# de Macao, de Taiwan et de la la Chine continentale, donc :

# Restriction : suppression des lignes "Chine" dans tous les tableaux :
doublons = ["Chine"]
fr_animaux.drop(fr_animaux.loc[fr_animaux["Zone"].isin(doublons)].index, inplace=True)
fr_cereales.drop(fr_cereales.loc[fr_cereales["Zone"].isin(doublons)].index, inplace=True)
fr_population.drop(fr_population.loc[fr_population["Zone"].isin(doublons)].index, inplace=True)
fr_sousalimentation.drop(fr_sousalimentation.loc[fr_sousalimentation["Zone"].isin(doublons)].index, inplace=True)
fr_vegetaux.drop(fr_vegetaux.loc[fr_vegetaux["Zone"].isin(doublons)].index, inplace=True)

In [11]:
# Calcul du nouveau total :
populationMondiale = fr_population["Valeur"].sum()*1000
populationMondiale

6997326000

In [12]:
# QUESTION 2 -------------------------------------------------------------------------------------------------------------------
# Identifier les redondances sous forme d'une équation.

In [13]:
# Équations :
# Disponibilité intérieure = Production + Imports - Exports  - Variations de stock
# Nourriture = Disponibilité Intérieure - Traitement - Semences - Pertes - Aliments pour animaux - Autres utilisations
# Nourriture = Production + Imports + Exports - Variations de stock - Traitement - Semences - Pertes - Aliments pour animaux - Autres utilisations

In [14]:
# création d'un dataframe sur le blé en France à partir de "fr_vegetaux - Question 2" :
franceBle = fr_vegetaux.loc[fr_vegetaux["Zone"] == "France"]
franceBle = franceBle.loc[franceBle["Produit"] == "Blé"]

# Création d'un tableau pivot :
bleFrance = franceBle.pivot_table("Valeur", index="Élément")
bleFrance

,Valeur
Élément,
Aliments pour animaux,7822.00
Autres utilisations (non alimentaire),2824.00
Disponibilité alimentaire (Kcal/personne/jour),811.00
Disponibilité alimentaire en quantité (kg/personne/an),108.43
Disponibilité de matière grasse en quantité (g/personne/jour),4.20
Disponibilité de protéines en quantité (g/personne/jour),25.99
Disponibilité intérieure,20298.00
Exportations - Quantité,21502.00
Importations - Quantité,2055.00


In [15]:
# Vérification de l'égalité suivante :
# Disponibilité intérieure = Production + Importations - Exportations + Variations de stock
# Disponibilité intérieure indiquée : 20298.00

disponibilitéInt = bleFrance.Valeur["Production"] + bleFrance.Valeur["Importations - Quantité"] - bleFrance.Valeur["Exportations - Quantité"] + bleFrance.Valeur["Variation de stock"]
disponibilitéInt

20298.0

In [16]:
# Vérification de l'égalité suivante :
# Nourriture = Disponibilité Intérieure - Traitement - Semences - Pertes - Aliments pour animaux - Autres utilisations
# Nourriture indiquée : 6971.00

nourriture = bleFrance.Valeur["Disponibilité intérieure"] - bleFrance.Valeur["Traitement"] - bleFrance.Valeur["Semences"] - bleFrance.Valeur["Pertes"] - bleFrance.Valeur["Aliments pour animaux"] - bleFrance.Valeur["Autres utilisations (non alimentaire)"]
nourriture

6971.0

In [17]:
# Vérification de l'égalité suivante :
# Nourriture = Production + Imports + Exports + Variations de stock - Traitement - Semences - Pertes - Aliments pour animaux - Autres utilisations
nourriture2 = bleFrance.Valeur["Production"] + bleFrance.Valeur["Importations - Quantité"] - bleFrance.Valeur["Exportations - Quantité"] + bleFrance.Valeur["Variation de stock"]- bleFrance.Valeur["Traitement"] - bleFrance.Valeur["Semences"] - bleFrance.Valeur["Pertes"] - bleFrance.Valeur["Aliments pour animaux"] - bleFrance.Valeur["Autres utilisations (non alimentaire)"]
nourriture2

6971.0

In [18]:
# QUESTION 3 -------------------------------------------------------------------------------------------------------------------
# Calculer (pour chaque pays et chaque produit) la disponibilité en Kcal puis en Kg de protéines

In [19]:
# Préparation du dataframe fr_population en vue d'une jointure = fr_population 2.

# Conversion de l'unité "Population en milliers" en "Population" :
fr_population["Population"]=fr_population["Valeur"]*1000

# Suppression des colonnes inutiles :
fr_population2 = fr_population.drop(columns=["Code Domaine","Domaine","Code zone","Code Élément","Élément", "Code Produit","Produit","Code année","Année","Unité", "Symbole","Valeur","Description du Symbole"])

In [20]:
# Union et jointure des différents dataframe = fr_alimentation3.

# Union des données :
fr_alimentation = pd.concat([fr_animaux, fr_vegetaux])

# Jointure avec les données de populations :
fr_alimentation2 = pd.merge(fr_alimentation, fr_population2, how="left")

# suppression des colonnes inutiles :
fr_alimentation3 = fr_alimentation2.drop(columns=["Code Domaine","Domaine","Code zone","Code Élément","Code Produit","Code année","Année","Unité", "Symbole","Description du Symbole"])
fr_alimentation3.head()

,Zone,Élément,Produit,Valeur,Population
0,Afghanistan,Production,Viande de Bovins,134.00,30552000
1,Afghanistan,Importations - Quantité,Viande de Bovins,6.00,30552000
2,Afghanistan,Disponibilité intérieure,Viande de Bovins,140.00,30552000
3,Afghanistan,Nourriture,Viande de Bovins,140.00,30552000
4,Afghanistan,Disponibilité alimentaire en quantité (kg/pers...,Viande de Bovins,4.59,30552000


In [21]:
# Création d'un masque "Disponibilité alimentaire en Kcal" :

# Restriction :
dispoKcal = fr_alimentation3.loc[fr_alimentation3["Élément"] == "Disponibilité alimentaire (Kcal/personne/jour)"].drop(columns=["Élément"])

# Conversion des données :
dispoKcal["Dispo en Kcal"]=dispoKcal["Valeur"]*dispoKcal["Population"]*365
dispoKcal = dispoKcal.drop(columns=["Valeur"])
dispoKcal.head()

,Zone,Produit,Population,Dispo en Kcal
5,Afghanistan,Viande de Bovins,30552000,3.010900e+11
12,Afghanistan,Viande d'Ovins/Caprins,30552000,3.456959e+11
20,Afghanistan,Viande de Volailles,30552000,1.003633e+11
28,Afghanistan,"Viande, Autre",30552000,1.115148e+10
35,Afghanistan,Abats Comestible,30552000,5.575740e+10


In [22]:
# Création d'un masque "Disponibilité alimentaire en Kg de protéines" :

# Restriction :
dispoProteines = fr_alimentation3.loc[fr_alimentation3["Élément"] == "Disponibilité de protéines en quantité (g/personne/jour)"].drop(columns=["Élément"])

# Conversion des données :
dispoProteines["Dispo protéines en Kg"]=dispoProteines["Valeur"]*dispoProteines["Population"]*365/1000
dispoProteines = dispoProteines.drop(columns=["Valeur"])
dispoProteines.head()

,Zone,Produit,Population,Dispo protéines en Kg
6,Afghanistan,Viande de Bovins,30552000,21076297.2
13,Afghanistan,Viande d'Ovins/Caprins,30552000,21299326.8
21,Afghanistan,Viande de Volailles,30552000,9701787.6
29,Afghanistan,"Viande, Autre",30552000,1895751.6
36,Afghanistan,Abats Comestible,30552000,8586639.6


In [23]:
# Tableau pivot pour la disponibilité en Kcal pour chaque pays et chaque produit :
dispoPaysProduitKcal = dispoKcal.pivot_table(values="Dispo en Kcal" , index="Zone", columns="Produit")
dispoPaysProduitKcal.head()

Produit,Abats Comestible,"Agrumes, Autres",Aliments pour enfants,Ananas,Animaux Aquatiques Autre,Arachides Decortiquees,Avoine,Bananes,Bananes plantains,"Beurre, Ghee",...,Thé,Tomates,Viande d'Ovins/Caprins,Viande de Anim Aquatiq,Viande de Bovins,Viande de Suides,Viande de Volailles,"Viande, Autre",Vin,"Épices, Autres"
Zone,,,,,,,,,,,,,,,,,,,,,
Afghanistan,5.575740e+10,1.115148e+10,1.115148e+10,0.000000e+00,NaN,NaN,NaN,4.460592e+10,NaN,2.564840e+11,...,4.460592e+10,0.000000e+00,3.456959e+11,NaN,3.010900e+11,NaN,1.003633e+11,1.115148e+10,0.000000e+00,2.230296e+10
Afrique du Sud,3.274751e+11,0.000000e+00,0.000000e+00,1.926324e+10,0.0,2.118956e+11,9.631620e+10,2.311589e+11,5.778972e+10,1.348427e+11,...,0.000000e+00,9.631620e+10,4.815810e+11,NaN,2.138220e+12,8.090561e+11,2.908749e+12,5.778972e+10,2.696854e+11,1.926324e+10
Albanie,1.273960e+10,0.000000e+00,1.158145e+09,0.000000e+00,0.0,2.316290e+09,1.158145e+09,1.158145e+10,0.000000e+00,2.663734e+10,...,0.000000e+00,3.242806e+10,9.844232e+10,NaN,1.169726e+11,9.612604e+10,7.759572e+10,0.000000e+00,1.737218e+10,0.000000e+00
Algérie,5.724368e+10,0.000000e+00,8.586552e+10,0.000000e+00,0.0,7.155460e+10,0.000000e+00,1.431092e+11,0.000000e+00,1.574201e+11,...,0.000000e+00,2.289747e+11,6.153696e+11,NaN,3.577730e+11,0.000000e+00,3.720839e+11,1.431092e+10,0.000000e+00,2.862184e+10
Allemagne,6.039071e+10,3.019536e+10,0.000000e+00,1.509768e+11,0.0,6.039071e+11,3.925396e+11,6.039071e+11,0.000000e+00,3.049731e+12,...,3.019536e+10,3.321489e+11,1.509768e+11,NaN,1.177619e+12,7.156299e+12,1.962698e+12,2.113675e+11,1.298400e+12,1.207814e+11


In [24]:
# Tableau pivot pour la disponibilité pour chaque pays et chaque produit en Kg de protéines :
dispoPaysProduitKgProteines = dispoProteines.pivot_table(values="Dispo protéines en Kg", index="Zone", columns="Produit")
dispoPaysProduitKgProteines.head()

Produit,Abats Comestible,"Agrumes, Autres",Aliments pour enfants,Ananas,Animaux Aquatiques Autre,Arachides Decortiquees,Avoine,Bananes,Bananes plantains,"Beurre, Ghee",...,Thé,Tomates,Viande d'Ovins/Caprins,Viande de Anim Aquatiq,Viande de Bovins,Viande de Suides,Viande de Volailles,"Viande, Autre",Vin,"Épices, Autres"
Zone,,,,,,,,,,,,,,,,,,,,,
Afghanistan,8586639.60,223029.60,334544.4,NaN,NaN,NaN,NaN,557574.0,NaN,334544.40,...,10705420.80,0.00,21299326.80,NaN,2.107630e+07,NaN,9701787.6,1895751.6,NaN,1226662.80
Afrique du Sud,49699159.20,0.00,0.0,192632.40,0.0,9246355.20,3082118.40,3660015.6,770529.6,192632.40,...,1926324.00,5201074.80,29472757.20,NaN,1.392732e+08,2.138220e+07,286059114.0,8475825.6,0.0,963162.00
Albanie,1853032.00,0.00,46325.8,NaN,0.0,127395.95,34744.35,138977.4,NaN,34744.35,...,11581.45,1540332.85,6555100.70,NaN,1.084024e+07,3.161736e+06,5721236.3,23162.9,NaN,11581.45
Algérie,9015879.60,0.00,3291511.6,0.00,0.0,3005293.20,0.00,2432856.4,0.0,143109.20,...,1287982.80,11305626.80,40213685.20,NaN,3.219957e+07,0.000000e+00,32628897.6,2146638.0,NaN,1287982.80
Allemagne,11172281.35,301953.55,0.0,905860.65,0.0,27175819.50,14493770.40,7850792.3,0.0,3925396.15,...,4227349.70,15399631.05,7548838.75,NaN,1.422201e+08,4.305858e+08,219218277.3,42273497.0,0.0,3925396.15


In [25]:
# Exemple 1 : Disponibilité en Kcal pour le riz au Brésil.
dispoPaysProduitKcal["Riz (Eq Blanchi)"]["Brésil"]

23841074380000.0

In [26]:
# Exemple 2 : Disponibilité en Kilos de Protéines pour le blé en France.
dispoPaysProduitKgProteines["Blé"]["France"]

609886927.85

In [27]:
# QUESTION 4 -------------------------------------------------------------------------------------------------------------------
# Calculer pour (chaque produit et chaque pays) le ratio/énergie poids en Kcal puis en pourcentage de protéines.
# Indication : kcal/personne/jour = Ratio énergie/poids * Nourriture

In [28]:
### 1 - Création d'un masque "Disponibilité Nourriture en kg"

# Restriction
dispoNourriture = fr_alimentation3.loc[fr_alimentation3["Élément"] == "Nourriture"].drop(columns=["Élément","Population"])

# Conversion des données
dispoNourriture["Valeur en Kilos"]=dispoNourriture["Valeur"]*1000000
dispoNourriture = dispoNourriture.drop(columns=["Valeur"])
dispoNourriture = dispoNourriture.replace(0, np.nan)
dispoNourriture.head()

,Zone,Produit,Valeur en Kilos
3,Afghanistan,Viande de Bovins,140000000.0
10,Afghanistan,Viande d'Ovins/Caprins,150000000.0
18,Afghanistan,Viande de Volailles,75000000.0
26,Afghanistan,"Viande, Autre",12000000.0
33,Afghanistan,Abats Comestible,53000000.0


In [29]:
# Jointure entre les données en Kcal et celles en kilos :
ratioKcalKg = pd.merge(dispoKcal, dispoNourriture, how="left").drop(columns=["Population"])
ratioKcalKg["Ratio énergie/poids (en kcal/kg)"] = ratioKcalKg["Dispo en Kcal"]/ratioKcalKg["Valeur en Kilos"]
ratioKcalKg.head()

,Zone,Produit,Dispo en Kcal,Valeur en Kilos,Ratio énergie/poids (en kcal/kg)
0,Afghanistan,Viande de Bovins,3.010900e+11,140000000.0,2150.642571
1,Afghanistan,Viande d'Ovins/Caprins,3.456959e+11,150000000.0,2304.639200
2,Afghanistan,Viande de Volailles,1.003633e+11,75000000.0,1338.177600
3,Afghanistan,"Viande, Autre",1.115148e+10,12000000.0,929.290000
4,Afghanistan,Abats Comestible,5.575740e+10,53000000.0,1052.026415


In [30]:
# Tableau pivot indiquant le ratio énergie/poids pour chaque produit :
ratioEP = ratioKcalKg.pivot_table(values="Ratio énergie/poids (en kcal/kg)", index="Zone", columns="Produit")
ratioEP.head()

Produit,Abats Comestible,"Agrumes, Autres",Aliments pour enfants,Ananas,Animaux Aquatiques Autre,Arachides Decortiquees,Avoine,Bananes,Bananes plantains,"Beurre, Ghee",...,Sésame,Thé,Tomates,Viande d'Ovins/Caprins,Viande de Bovins,Viande de Suides,Viande de Volailles,"Viande, Autre",Vin,"Épices, Autres"
Zone,,,,,,,,,,,,,,,,,,,,,
Afghanistan,1052.026415,285.935385,5575.740000,NaN,NaN,NaN,NaN,543.974634,NaN,7124.556667,...,6272.7075,416.877757,NaN,2304.639200,2150.642571,NaN,1338.177600,929.290000,NaN,2787.870000
Afrique du Sud,1289.271969,0.000000,NaN,447.982326,NaN,5576.201053,2407.90500,623.069757,671.973488,7096.983158,...,NaN,0.000000,190.348221,2130.889381,2246.029034,3677.527636,1466.842784,1256.298261,662.617592,2751.891429
Albanie,1158.145000,0.000000,NaN,NaN,NaN,NaN,NaN,579.072500,0.000000,6659.333750,...,NaN,NaN,175.286811,2009.027041,1647.502042,2746.458143,1847.517024,NaN,694.887000,NaN
Algérie,1004.275088,0.000000,3902.978182,0.000000,NaN,6504.963636,NaN,579.389474,NaN,7871.006000,...,5724.3680,0.000000,213.396757,2044.417143,1626.240909,NaN,1278.638900,1022.208571,NaN,2862.184000
Allemagne,974.043710,160.613590,NaN,553.028480,NaN,5751.496190,3090.86311,616.231735,0.000000,7142.226827,...,NaN,511.785678,211.156329,2516.279583,1081.376350,1669.691819,1336.077655,1031.060902,684.810266,3552.394706


In [31]:
# Vérification de la cohérence de la valeur obtenue avec l'oeuf pour exemple,
# Wikipédia indiquant une valeur calorique de 1470 kcal pour 1kg.
ratioEP["Oeufs"]["France"]

1417.9821860189572

In [32]:
# Jointure entre les données de Protéines et les quantités en kilos :
ratioProteinesKg = pd.merge(dispoNourriture, dispoProteines, how="left").drop(columns=["Population"])
ratioProteinesKg["Rapport protéines/poids (en %)"] = ratioProteinesKg["Dispo protéines en Kg"]/ratioProteinesKg["Valeur en Kilos"]*100
ratioProteinesKg.head()

,Zone,Produit,Valeur en Kilos,Dispo protéines en Kg,Rapport protéines/poids (en %)
0,Afghanistan,Viande de Bovins,140000000.0,21076297.2,15.054498
1,Afghanistan,Viande d'Ovins/Caprins,150000000.0,21299326.8,14.199551
2,Afghanistan,Viande de Volailles,75000000.0,9701787.6,12.935717
3,Afghanistan,"Viande, Autre",12000000.0,1895751.6,15.797930
4,Afghanistan,Abats Comestible,53000000.0,8586639.6,16.201207


In [33]:
# Tableau pivot indiquant le rapport protéines/poids pour chaque produit :
ratioPP = ratioProteinesKg.pivot_table(values="Rapport protéines/poids (en %)", index="Zone", columns="Produit")
ratioPP = ratioPP.replace(0, np.NaN)
ratioPP.loc["Moyenne"] = ratioPP.mean(axis=0)
ratioPP.tail()

Produit,Abats Comestible,"Agrumes, Autres",Aliments pour enfants,Ananas,Animaux Aquatiques Autre,Arachides Decortiquees,Avoine,Bananes,Bananes plantains,"Beurre, Ghee",...,Sésame,Thé,Tomates,Viande d'Ovins/Caprins,Viande de Bovins,Viande de Suides,Viande de Volailles,"Viande, Autre",Vin,"Épices, Autres"
Zone,,,,,,,,,,,,,,,,,,,,,
Équateur,18.425338,0.499510,14.360925,0.182361,NaN,26.070602,9.533210,0.729607,0.700905,NaN,...,NaN,11.488740,1.025780,13.950613,13.931748,9.793265,11.622330,16.658673,NaN,13.403530
États-Unis d'Amérique,18.098659,NaN,NaN,0.287447,NaN,27.479735,7.295392,0.715533,0.800127,0.857699,...,18.841712,8.444719,0.802644,13.446747,12.368793,9.784401,13.093377,18.041485,NaN,9.734885
Éthiopie,16.896442,NaN,22.897910,NaN,NaN,26.420665,11.448955,0.648054,NaN,NaN,...,17.173433,9.813390,1.073340,13.692331,14.748948,17.173433,12.187597,16.717501,NaN,11.448955
Îles Salomon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,17.882810,9.623955,11.671605,NaN,NaN,NaN
Moyenne,17.561047,0.442631,14.883228,0.307905,3.889703,25.146613,8.447081,0.821462,0.771368,0.900611,...,17.503953,9.111332,0.956777,13.580332,14.378022,10.297519,12.620941,17.402372,0.007165,10.741790


In [34]:
# Vérification de la cohérence de la valeur obtenue avec l'avoine pour exemple :
ratioPP["Avoine"]["France"]

8.244886351351353

In [35]:
# Wikipédia indique une valeur calorique de 10,7/100g.
# Le résultat est un peu en dessous mais reste cohérent, d'autant que la même fonction appliquée à l'Allemagne donnent 11,4 %. 

In [36]:
# QUESTION 5 -------------------------------------------------------------------------------------------------------------------
# Citez 5 aliments parmi les 20 plus caloriques en utilisant un ratio énergie/poids

In [37]:
# Création d'un dataframe d'agrégation :

# Remplacement des "0" par "NaN" :
ratioPP = ratioPP.replace(0, np.NaN)
ratioEP = ratioEP.replace(0, np.NaN)

# Moyenne :
ratioPP.loc["Moyenne"] = ratioPP.mean(axis=0)
ratioEP.loc["Moyenne"] = ratioEP.mean(axis=0)

In [38]:
# Afficher les 20 aliments les plus caloriques en termes de Kcal.
ratioEP.loc["Moyenne"].sort_values(axis=0,ascending=False).head(20)

Produit
Huiles de Foie de Poisso    9982.932500
Huile de Sésame             9433.744729
Huile de Son de Riz         9361.549883
Huiles de Poissons          9003.715504
Huile d'Arachide            8935.765811
Huile d'Olive               8931.223447
Huile de Germe de Maïs      8871.762566
Huil Plantes Oleif Autr     8848.222893
Huile de Palmistes          8789.540277
Huile Graines de Coton      8750.320136
Huile de Palme              8719.419872
Huile de Tournesol          8693.245772
Huile de Soja               8635.808245
Huile de Colza&Moutarde     8627.372452
Huile de Coco               8565.739255
Graisses Animales Crue      7583.862063
Beurre, Ghee                7331.424291
Palmistes                   6496.635000
Sésame                      5796.120413
Arachides Decortiquees      5702.900100
Name: Moyenne, dtype: float64

In [39]:
# Afficher les 20 aliments les plus caloriques en termes de protéines.
ratioPP.loc["Moyenne"].sort_values(axis=0,ascending=False).head(20)

Produit
Graines Colza/Moutarde      28.536511
Soja                        28.533053
Arachides Decortiquees      25.146613
Pois                        22.324375
Légumineuses Autres         21.971635
Haricots                    21.545181
Abats Comestible            17.561047
Sésame                      17.503953
Viande, Autre               17.402372
Plantes Oleiferes, Autre    17.054422
Aliments pour enfants       14.883228
Viande de Bovins            14.378022
Cephalopodes                13.990390
Viande d'Ovins/Caprins      13.580332
Viande de Volailles         12.620941
Piments                     12.100347
Poissons Pelagiques         11.934551
Graines de tournesol        11.182268
Poissons Marins, Autres     11.013261
Épices, Autres              10.741790
Name: Moyenne, dtype: float64

In [40]:
# QUESTION 6 -------------------------------------------------------------------------------------------------------------------
# Calculer pour les produits végétaux uniquement la disponibilité mondiale exprimée en kcal.

In [41]:
# 1 - Création d'un tableau de valeurs des produits en Kcal/an :

# Restriction sur la disponibilité en Kcal/personne/jour :
vegKcal = fr_vegetaux.loc[fr_vegetaux["Élément"] == "Disponibilité alimentaire (Kcal/personne/jour)"].drop(columns=["Élément"])

# Jointure avec les données de population :
vegKcal = pd.merge(vegKcal, fr_population2, how="left")

# Conversion des unités :
vegKcal["Valeur en kcal"] = vegKcal["Valeur"]*vegKcal["Population"]*365

# Calcul du Total de disponibilité alimentaire en Kcal
vegTotalKcal = vegKcal["Valeur en kcal"].sum(axis=0)
vegTotalKcal

6051609427075000.0

In [42]:
# 2 - Création d'un tableau de valeurs de quantité de nourriture en Kg de nourriture par an

# Restriction sur la quantité de nourriture :
vegQuantiteKg = fr_vegetaux.loc[fr_vegetaux["Élément"] == "Nourriture"].drop(columns=["Élément"])

# Conversion des unités en Kg/an :
vegQuantiteKg["Valeur en kg"] = vegQuantiteKg["Valeur"]*1000000

# Calcul du Total de disponibilité alimentaire en Kg :
vegTotalQuantiteKg = vegQuantiteKg["Valeur en kg"].sum(axis=0)
vegTotalQuantiteKg

3692542000000.0

In [43]:
# 3 - Extraction d'un ratio Kcal/Kg
vegRatioKcalKg = vegTotalKcal/vegTotalQuantiteKg
vegRatioKcalKg

1638.8735529819296

In [44]:
# 4 - Création d'un tableau de disponibilité alimentaire intérieure (en Kg)

# Restriction sur la disponibilité intérieure :
vegDispoInt = fr_vegetaux.loc[fr_vegetaux["Élément"] == "Disponibilité intérieure"].drop(columns=["Élément"])

# Conversion des unités en Kg :
vegDispoInt["Valeur"] = vegDispoInt["Valeur"]*1000000

# Calcul du Total de disponibilité intérieure en Kg :
vegTotalDispoInt = vegDispoInt["Valeur"].sum(axis=0)
vegTotalDispoInt

8482244000000.0

In [45]:
# 5 - Calcul de la disponibilité intérieure Mondiale en végétaux en terme de Kcal :
vegDispoIntMondialeKcal = vegTotalDispoInt*vegRatioKcalKg
vegDispoIntMondialeKcal

1.3901325361539654e+16

In [46]:
# QUESTION 7 -------------------------------------------------------------------------------------------------------------------
# Combien d'humains pourraient être nourris si toute la disponibilité intérieure mondiale était utilisée pour de la nourriture (en termes de Kcal et de protéines) ?

In [47]:
# les recommandations alimentaires trouvés sur internet sont de l'ordre de 2500 Kcal.
# J'effectuerai donc les calculs selon cette valeur.

In [48]:
# Calcul du nombre d'individus pouvant être nourris avec la disponibilité mondiale en végétaux en terme de Kcal :
besoinsKcalAn = 2500*365
popNourrissableDispoKcal = vegDispoIntMondialeKcal/besoinsKcalAn
popNourrissableDispoKcal

15234329163.331127

In [49]:
# La seule disponibilité intérieure mondiale en végétaux pourrait nourrir plus de 15 milliards de personnes.

In [50]:
###### Calcul de la disponibilité mondiale en terme de protéines végétales :

### 1 - Création d'un tableau de valeur des produits en kg de protéines par an

# Restriction sur la disponibilité en grammes de protéines/personne/jour :
vegProt = fr_vegetaux.loc[fr_vegetaux["Élément"] == "Disponibilité de protéines en quantité (g/personne/jour)"].drop(columns=["Élément"])

# Jointure avec les données de population :
vegProt = pd.merge(vegProt, fr_population2, how="left")

# Conversion des unités en Kg de protéines par an :
vegProt["Valeur (en kg/an)"] = vegProt["Valeur"]*vegProt["Population"]*365/1000

# Calcul du total de disponibilité en kg de protéines :
vegProt = vegProt["Valeur (en kg/an)"].sum(axis=0)

### 2 - Extraction d'un ratio Kg de protéines/Kg
vegRatioProteinesKg = vegProt/vegTotalQuantiteKg
vegRatioProteinesKg

### 3 - Calcul de la disponibilité intérieure Mondiale en végétaux en terme de protéines :
vegDispoIntMondialeProt = vegTotalDispoInt*vegRatioProteinesKg
vegDispoIntMondialeProt

287604279385.0921

In [51]:
# L'ANSES recommande un apport en protéine minimum de 0,83 g/kg/jour (poids moyen d'un humain à 62 kg).
# J'effectuerai donc les calculs suivants selon cette valeur.

In [52]:
# Calcul du nombre d'individus pouvant être nourris avec la disponibilité mondiale en végétaux en terme de Protéines :
besoinsProt = 0.83*365*62/1000
popNourrissableDispoProt = vegDispoIntMondialeProt/besoinsProt
popNourrissableDispoProt

15312027396.466581

In [53]:
# Calcul des pourcentages de la population mondiale pouvant être nourrie par la seule disponibilité alimentaire végétale :
vegDispoIntMondialeKcal = popNourrissableDispoKcal/populationMondiale*100
vegDispoIntMondialeProt = popNourrissableDispoProt/populationMondiale*100

In [54]:
# Pourcentage de la population pouvant être nourrie avec la disponibilité mondiale de végétaux en terme de Kcal en 2013 :
vegDispoIntMondialeKcal

217.71644144250425

In [55]:
# Pourcentage de la population pouvant être nourrie avec la disponibilité mondiale de végétaux en terme de protéines en 2013 :
vegDispoIntMondialeProt

218.82684037397402

In [56]:
# QUESTION 8 -------------------------------------------------------------------------------------------------------------------

In [57]:
# 1 - Calcul du total cumulé de la disponibilité intérieure, de la nourritures pour animaux et des pertes pour les végétaux :
vegDispoIntAnimauxPertes1 = fr_vegetaux.loc[fr_vegetaux["Élément"] == "Nourriture"]
vegDispoIntAnimauxPertes2 = fr_vegetaux.loc[fr_vegetaux["Élément"] == "Pertes"]
vegDispoIntAnimauxPertes3 = fr_vegetaux.loc[fr_vegetaux["Élément"] == "Aliments pour animaux"]
vegDispoIntAnimauxPertes1T = vegDispoIntAnimauxPertes1["Valeur"].sum(axis=0)*1000000
vegDispoIntAnimauxPertes2T = vegDispoIntAnimauxPertes2["Valeur"].sum(axis=0)*1000000
vegDispoIntAnimauxPertes3T = vegDispoIntAnimauxPertes3["Valeur"].sum(axis=0)*1000000
vegDispoIntAnimauxPertesT = vegDispoIntAnimauxPertes1T + vegDispoIntAnimauxPertes2T + vegDispoIntAnimauxPertes3T
vegDispoIntAnimauxPertesT

5318929000000.0

In [58]:
# Calcul de disponibilité mondiale végétale + Pertes + Aliments pour animaux en termes de Kcal :
vegTotalDispoIntAnimauxPertesKcal = vegDispoIntAnimauxPertesT * vegRatioKcalKg
vegTotalDispoIntAnimauxPertesKcal

8717052068288622.0

In [59]:
# Calcul de disponibilité mondiale végétale + Pertes + Aliments pour animaux en termes de Protéines :
vegTotalDispoIntAnimauxPertesProt = vegDispoIntAnimauxPertesT * vegRatioProteinesKg
vegTotalDispoIntAnimauxPertesProt

180346939105.4382

In [60]:
# Calcul du nombre d'individus pouvant être nourris avec la disponibilité mondiale (+V, +AA) en terme de Kcal :
popNourrissableDispoIntAnimauxPertesKcal = vegTotalDispoIntAnimauxPertesKcal/besoinsKcalAn
popNourrissableDispoIntAnimauxPertesKcal

9552933773.466984

In [61]:
# Calcul du nombre d'individus pouvant être nourris avec la disponibilité mondiale en végétaux (+V, +AA) en terme de protéines :
popNourrissableDispoIntAnimauxPertesProt = vegTotalDispoIntAnimauxPertesProt/besoinsProt
popNourrissableDispoIntAnimauxPertesProt

9601655713.731012

In [62]:
# Pourcentage de la population pouvant être nourrie en terme de Kcal en 2013 :
vegDispoIntAnimauxPertesMondialeKcal = popNourrissableDispoIntAnimauxPertesKcal/populationMondiale*100
vegDispoIntAnimauxPertesMondialeKcal

136.52263412433527

In [63]:
# Pourcentage de la population pouvant être nourrie en terme de protéines en 2013 :
vegDispoIntAnimauxPertesMondialeProt = popNourrissableDispoIntAnimauxPertesProt/populationMondiale*100
vegDispoIntAnimauxPertesMondialeProt

137.21892782658708

In [64]:
# QUESTION 9 -------------------------------------------------------------------------------------------------------------------

In [65]:
###  1 - Calcul du total de la disponibilité mondiale en kg:

# Restriction sur la nourriture en milliers de tonnes :
alimDispoMondiale = fr_alimentation3.loc[fr_alimentation3["Élément"] == "Nourriture"]

# Conversion des unités en kg :
alimDispoMondiale = alimDispoMondiale["Valeur"].sum(axis=0)*1000000
alimDispoMondiale

4876258000000.0

In [66]:
### 2 - Valeurs des produits en Kcal :

# Restriction sur la disponibilité en Kcal/personne/jour :
alimKcal = fr_alimentation3.loc[fr_alimentation3["Élément"] == "Disponibilité alimentaire (Kcal/personne/jour)"].drop(columns=["Élément"])

# Conversion des unités  en Kcal :
alimKcal["Valeur en kcal"] = alimKcal["Valeur"]*alimKcal["Population"]*365

# Calcul du Total de disponibilité alimentaire en Kcal
alimKcal = alimKcal["Valeur en kcal"].sum(axis=0)
alimKcal

7363883420185000.0

In [67]:
### 3 - Valeurs des produits en protéines :

# Restriction sur la disponibilité en grammes de protéines/personne/jour :
alimProt = fr_alimentation3.loc[fr_alimentation3["Élément"] == "Disponibilité de protéines en quantité (g/personne/jour)"].drop(columns=["Élément"])

# Conversion des unités en kg de protéines :
alimProt["Valeur en kg"] = alimProt["Valeur"]*alimProt["Population"]*365/1000

# Calcul du Total de disponibilité alimentaire en kg de protéines :
alimProt = alimProt["Valeur en kg"].sum(axis=0)
alimProt

207259180075.55

In [68]:
### 4 - Valeur Kg de nourriture :

# Calcul de la disponibilité en Kg/an :
alimKg = fr_alimentation3.loc[fr_alimentation3["Élément"] == "Nourriture"].drop(columns=["Élément"])
alimKg = alimKg["Valeur"].sum()*1000000
alimKg

4876258000000.0

In [69]:
### 5 - Extraction des ratios :

# ratio en kcal :
alimRatioKcalKg = alimKcal/alimKg

# ratio en protéines
alimRatioProtKg = alimProt/alimKg

In [70]:
# Calcul du nombre d'individus pouvant être nourris avec la disponibilité mondiale en termes de Kcal :
popNourrissableDispoMondialeKcal = alimDispoMondiale * alimRatioKcalKg
popNourrissableDispoMondialeKcal = popNourrissableDispoMondialeKcal/besoinsKcalAn
popNourrissableDispoMondialeKcal

8070009227.6

In [71]:
# Calcul du nombre d'individus pouvant être nourris avec la disponibilité mondiale en termes de protéines :
popNourrissableDispoMondialeProt = alimDispoMondiale * alimRatioProtKg
popNourrissableDispoMondialeProt = popNourrissableDispoMondialeProt/besoinsProt
popNourrissableDispoMondialeProt

11034461136.222311

In [72]:
# Pourcentage de la population pouvant être nourrie avec la totalité de la disponibilité mondiale terme de Kcal en 2013 :
alimDispoMondialeKcal = popNourrissableDispoMondialeKcal/populationMondiale*100
alimDispoMondialeKcal

115.32990213118555

In [73]:
# Pourcentage de la population pouvant être nourrie avec la totalité de la disponibilité mondiale terme de protéines en 2013 :
alimDispoMondialeProt = popNourrissableDispoMondialeProt/populationMondiale*100
alimDispoMondialeProt

157.69539873120547

In [74]:
# QUESTION 10 ------------------------------------------------------------------------------------------------------------------
# Quelle proportion de la population mondiale est considérée comme étant en sous-nutrition ?

In [75]:
# Création d'un dataframe à partir de "fr_population.csv" :

# Conversion des valeurs (car la présence de "<0.1" dans les données de valeur avait entraîné la transformation de toutes les données numérique en données alphabétiques).
fr_sousalimentation["Valeur"] = fr_sousalimentation["Valeur"].replace(to_replace = "<0.1", value = "0")
fr_sousalimentation["Valeur"] = fr_sousalimentation["Valeur"].astype(np.float)
fr_sousalimentation["Valeur"] = fr_sousalimentation["Valeur"]*1000000

# Jointure avec les données de population :
fr_sousalimentation2 = pd.merge(fr_sousalimentation, fr_population2, how="left")

# Restriction sur les années 2012 - 2014 :
    # Note : mes données de population datent de 2013, or, ce dataframe
    # indique une moyenne sur 3 années. J'ai choisi d'utiliser les données
    # 2012 - 2014, soit la moyenne de 2013 et de l'année -1 et l'année +1.
fr_sousalimentation3 = fr_sousalimentation2.loc[fr_sousalimentation2["Année"] == "2012-2014"]

# Suppression des colonnes inutiles (projection) :
fr_sousalimentation3 = fr_sousalimentation3.drop(columns=["Code Domaine","Domaine","Code zone","Code Élément","Élément","Code Produit","Produit","Code année","Année","Unité","Symbole","Description du Symbole","Note"])

# Création d'une colonne pourcentage de pourcentage de la population en situation de sous-alimentation.
fr_sousalimentation3["% de la population en sous-alimentation"] = fr_sousalimentation3["Valeur"]/fr_sousalimentation3["Population"]*100
fr_sousalimentation3.head()

,Zone,Valeur,Population,% de la population en sous-alimentation
0,Afghanistan,7900000.0,30552000.0,25.857554
5,Afrique du Sud,2600000.0,52776000.0,4.926482
10,Albanie,200000.0,3173000.0,6.303183
15,Algérie,1700000.0,39208000.0,4.335850
20,Allemagne,NaN,82727000.0,NaN


In [76]:
# Calcul de la proportion de la population mondiale considérée comme étant en sous-nutrition :
fr_sousalimentation3["Valeur"].sum() / fr_sousalimentation3["Population"].sum() * 100

10.628345742359295

In [77]:
# QUESTION 11 ------------------------------------------------------------------------------------------------------------------
# Quelle proportion de céréales est destinée à l'alimentation animale ?

In [78]:
# Liste des produits considérés comme céréales :
fr_cereales["Produit"].unique()

array(['Blé', 'Riz (Eq Blanchi)', 'Orge', 'Maïs', 'Millet', 'Seigle',
       'Avoine', 'Sorgho', 'Céréales, Autres'], dtype=object)

In [79]:
# Liste des codes des produits considérés comme céréales :
fr_cereales["Code Produit"].unique()

array([2511, 2805, 2513, 2514, 2517, 2515, 2516, 2518, 2520], dtype=int64)

In [80]:
# Jointure entre les données de fr_alimentation3 et fr_sousalimentation3 :

# Préparation de fr_sousalimentation4 en vue de la jointure :
fr_sousalimentation4 = fr_sousalimentation3.drop(columns=["Population"])
fr_sousalimentation4 = fr_sousalimentation4.rename(columns={"Valeur":"Nbre de personnes en sous-nutrition"})

# Jointure :
analyseSousNutrition = pd.merge(fr_alimentation3, fr_sousalimentation4, how="left")
analyseSousNutrition.head()

,Zone,Élément,Produit,Valeur,Population,Nbre de personnes en sous-nutrition,% de la population en sous-alimentation
0,Afghanistan,Production,Viande de Bovins,134.00,30552000,7900000.0,25.857554
1,Afghanistan,Importations - Quantité,Viande de Bovins,6.00,30552000,7900000.0,25.857554
2,Afghanistan,Disponibilité intérieure,Viande de Bovins,140.00,30552000,7900000.0,25.857554
3,Afghanistan,Nourriture,Viande de Bovins,140.00,30552000,7900000.0,25.857554
4,Afghanistan,Disponibilité alimentaire en quantité (kg/pers...,Viande de Bovins,4.59,30552000,7900000.0,25.857554


In [81]:
# Création d'un colonne de type boléen "Céréale"

# Liste des céréales :
cerealesFao = ["Blé", "Riz (Eq Blanchi)", "Orge", "Maïs", "Millet", "Seigle", "Avoine", "Sorgho", "Céréales, Autres"]

# Création de la colonne :
analyseSousNutrition["Céréales"] = analyseSousNutrition["Produit"].isin(cerealesFao)
analyseSousNutrition.head()

,Zone,Élément,Produit,Valeur,Population,Nbre de personnes en sous-nutrition,% de la population en sous-alimentation,Céréales
0,Afghanistan,Production,Viande de Bovins,134.00,30552000,7900000.0,25.857554,False
1,Afghanistan,Importations - Quantité,Viande de Bovins,6.00,30552000,7900000.0,25.857554,False
2,Afghanistan,Disponibilité intérieure,Viande de Bovins,140.00,30552000,7900000.0,25.857554,False
3,Afghanistan,Nourriture,Viande de Bovins,140.00,30552000,7900000.0,25.857554,False
4,Afghanistan,Disponibilité alimentaire en quantité (kg/pers...,Viande de Bovins,4.59,30552000,7900000.0,25.857554,False


In [82]:
### Calcul de la proportion de céréales destinée à l'alimentation animale (en %):

# 1 - Restriction sur "céréales" :
analyseSousNutritionCereales = analyseSousNutrition[analyseSousNutrition["Céréales"] == True]

# 2 - Calcul du poids de l'alimentation en céréales pour l'alimentation animale
cerealesAnimauxPoids = analyseSousNutritionCereales[analyseSousNutritionCereales["Élément"] == "Aliments pour animaux"]
cerealesAnimauxPoids = cerealesAnimauxPoids["Valeur"].sum()

# 3 - Calcul du poids de l'alimentation en céréales pour l'alimentation humaine
cerealesNourriturePoids = analyseSousNutritionCereales[analyseSousNutritionCereales["Élément"] == "Nourriture"]
cerealesNourriturePoids = cerealesNourriturePoids["Valeur"].sum()

# 4 - Calcul de la proportion de céréales destinées à l'alimentation animale (en %)  
cerealesPropAnimaux = cerealesAnimauxPoids/(cerealesNourriturePoids + cerealesAnimauxPoids)*100
cerealesPropAnimaux

45.91402568664604

In [83]:
# QUESTION 12 ------------------------------------------------------------------------------------------------------------------
# Donnez les 3 produits qui ont la plus grande valeur pour chacun des 2 ratios "Autres utilisation"/"Disponibilité intérieure"
# et "Nourriture animale/quantité destinée à la nourriture" :

In [84]:
# Restriction sur les pays dans lesquels la FAO recense des personnes en sous-nutrition :
analyseSousNutrition2 = analyseSousNutrition.loc[analyseSousNutrition["Nbre de personnes en sous-nutrition"] >0]

In [85]:
### Afficher les 15 produits les plus exportés par ce groupe de pays :

# 1 - Restriction sur les "exportations" :
analyseSousNutrition2Export = analyseSousNutrition2.loc[analyseSousNutrition2["Élément"] == "Exportations - Quantité"]

# 2 - Création d'un tableau pivot :
analyseSousNutrition2Export = analyseSousNutrition2Export.pivot_table(values="Valeur", index="Zone", columns="Produit")

# 3 - Création d'une ligne de moyennes :
analyseSousNutrition2Export.loc["Moyenne"] = analyseSousNutrition2Export.mean(axis=0)
analyseSousNutrition2Export

# Afficher les 15 produits les plus exportés par les pays comptant des personnes en sous-nutrition :
analyseSousNutrition2Export.loc["Moyenne"].sort_values(axis=0,ascending=False).head(15)

Produit
Huile de Palme         557.650602
Manioc                 543.075758
Maïs                   445.741176
Riz (Eq Blanchi)       368.393258
Sucre Eq Brut          277.329670
Blé                    270.010870
Bananes                231.233766
Légumes, Autres        209.978947
Soja                   208.536232
Lait - Excl Beurre     164.613636
Poissons Pelagiques    157.241379
Fruits, Autres         156.881720
Tomates                101.147727
Pommes                 101.116883
Ananas                  87.767123
Name: Moyenne, dtype: float64

In [86]:
### Afficher les 200 plus grandes importations :

# 1 - Restriction sur les "importations" pour analyseSousNutrition :
analyseSousNutritionImport = analyseSousNutrition.loc[analyseSousNutrition["Élément"] == "Importations - Quantité"]

# 2 - Suppression de la colonne Céréales
analyseSousNutritionImport = analyseSousNutritionImport.drop(columns = ["Céréales"])

# 3 - Restriction sur les 15 produits les plus exportés par les pays comptant des personnes en sous-nutrition
produitsLesPlusExportes = ["Huile de Palme","Manioc","Maïs","Riz (Eq Blanchi)","Sucre Eq Brut","Blé","Bananes","Légumes, Autres","Soja","Lait - Excl Beurre","Fruits, Autres","Poissons Pelagiques","Pommes","Tomates","Ananas"]
analyseSousNutritionImport["Parmi les 15 produits les plus exportés"] = analyseSousNutritionImport["Produit"].isin(produitsLesPlusExportes)
analyseSousNutritionImport = analyseSousNutritionImport[analyseSousNutritionImport["Parmi les 15 produits les plus exportés"] == True]

# 4 - Sélectionner les 200 plus grandes importations des 15 produits les plus exportés par les pays comptant des personnes en sous-nutrition :
analyseSousNutritionImport = analyseSousNutritionImport.sort_values(by="Valeur", ascending=False)
analyseSousNutritionImport = analyseSousNutritionImport.iloc[0:199]
analyseSousNutritionImport = analyseSousNutritionImport.pivot_table("Valeur","Zone","Produit")
analyseSousNutritionImport.head()

Produit,Ananas,Bananes,Blé,"Fruits, Autres",Huile de Palme,Lait - Excl Beurre,"Légumes, Autres",Manioc,Maïs,Poissons Pelagiques,Pommes,Riz (Eq Blanchi),Soja,Sucre Eq Brut,Tomates
Zone,,,,,,,,,,,,,,,
Afghanistan,NaN,NaN,1173.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Afrique du Sud,NaN,NaN,1490.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1311.0,NaN,NaN,NaN
Algérie,NaN,NaN,6343.0,NaN,NaN,2581.0,NaN,NaN,3251.0,NaN,NaN,NaN,NaN,1836.0,NaN
Allemagne,NaN,1344.0,5421.0,2499.0,2456.0,8332.0,4378.0,NaN,2465.0,1246.0,1177.0,NaN,3620.0,NaN,1764.0
Arabie saoudite,NaN,NaN,2484.0,NaN,NaN,2134.0,NaN,NaN,2120.0,NaN,NaN,1222.0,NaN,1780.0,NaN


In [87]:
### Calculer le ratio entre la quantité destinés aux "Autres utilisations" et la disponibilité intérieure pour ces produits.

## 1 - Calcul pour "Autres utilisations"

# a) Restriction sur les "Autres utilisations" pour analyseSousNutritionAutres :
analyseSousNutritionAutres = analyseSousNutrition.loc[analyseSousNutrition["Élément"] == "Autres utilisations (non alimentaire)"]

# b) Suppression de la colonne Céréales :
analyseSousNutritionAutres = analyseSousNutritionAutres.drop(columns = ["Céréales"])

# c) Restriction sur les 15 produits les plus exportés par les pays comptant des personnes en sous-nutrition :
analyseSousNutritionAutres["Parmi les 15 produits les plus exportés"] = analyseSousNutritionAutres["Produit"].isin(produitsLesPlusExportes)
analyseSousNutritionAutres = analyseSousNutritionAutres[analyseSousNutritionAutres["Parmi les 15 produits les plus exportés"] == True]

# d) Transformation en tableau Pivot
analyseSousNutritionAutres = analyseSousNutritionAutres.pivot_table("Valeur","Zone","Produit")
analyseSousNutritionAutres.loc["Moyenne"] = analyseSousNutritionAutres.mean()
analyseSousNutritionAutres["Global"] = analyseSousNutritionAutres.mean(axis=1)
analyseSousNutritionAutres.tail()

Produit,Ananas,Bananes,Blé,"Fruits, Autres",Huile de Palme,Lait - Excl Beurre,"Légumes, Autres",Manioc,Maïs,Poissons Pelagiques,Pommes,Riz (Eq Blanchi),Soja,Sucre Eq Brut,Tomates,Global
Zone,,,,,,,,,,,,,,,,
Équateur,NaN,NaN,3.000000,NaN,0.000000,1.000000,NaN,1.000000,633.000000,0.00000,NaN,151.000000,1.0,272.000000,NaN,118.000000
États-Unis d'Amérique,NaN,NaN,16.000000,NaN,1281.000000,5972.000000,NaN,245.000000,137023.000000,0.00000,NaN,142.000000,NaN,107.000000,NaN,18098.250000
Éthiopie,NaN,NaN,1600.000000,NaN,18.000000,0.000000,NaN,NaN,NaN,0.00000,NaN,NaN,NaN,NaN,NaN,404.500000
Îles Salomon,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,0.00000,NaN,12.000000,NaN,NaN,NaN,4.333333
Moyenne,0.428571,6.761905,167.343284,3.764706,208.230769,155.107438,13.611111,310.232759,1468.108527,2.17341,2.625,131.885714,63.8,121.574074,7.7,177.556485


In [88]:
### Calculer le ratio entre la quantité destinés aux "Autres utilisations" et la disponibilité intérieure pour ces produits.

## 2 - Calcul pour "disponibilité intérieure"

# a) Restriction sur les "Autres utilisations" :
analyseSousNutritionDI = analyseSousNutrition.loc[analyseSousNutrition["Élément"] == "Disponibilité intérieure"]

# b) Suppression de la colonne Céréales :
analyseSousNutritionDI = analyseSousNutritionDI.drop(columns = ["Céréales"])

# c) Création d'un masque pour les 15 produits les plus exportés par les pays comptant des personnes en sous-nutrition :
analyseSousNutritionDI["Parmi les 15 produits les plus exportés"] = analyseSousNutritionDI["Produit"].isin(produitsLesPlusExportes)
analyseSousNutritionDI = analyseSousNutritionDI[analyseSousNutritionDI["Parmi les 15 produits les plus exportés"] == True]

# d) Transformation en tableau Pivot
analyseSousNutritionDI = analyseSousNutritionDI.pivot_table("Valeur","Zone","Produit")
analyseSousNutritionDI.loc["Moyenne"] = analyseSousNutritionDI.mean()
analyseSousNutritionDI["Global"] = analyseSousNutritionDI.mean(axis=1)
analyseSousNutritionDI.tail()

Produit,Ananas,Bananes,Blé,"Fruits, Autres",Huile de Palme,Lait - Excl Beurre,"Légumes, Autres",Manioc,Maïs,Poissons Pelagiques,Pommes,Riz (Eq Blanchi),Soja,Sucre Eq Brut,Tomates,Global
Zone,,,,,,,,,,,,,,,,
Équateur,73.000000,584.0,623.00000,456.000000,120.000000,2367.000000,360.000000,44.00000,1690.000000,89.000000,69.000000,933.000000,87.000000,619.000000,30.000000,542.933333
États-Unis d'Amérique,2089.000000,3982.0,34051.00000,7688.000000,1304.000000,87965.000000,22911.000000,579.00000,292776.000000,1277.000000,6176.000000,3119.000000,52672.000000,10353.000000,12789.000000,35982.066667
Éthiopie,7.000000,295.0,4906.00000,318.000000,223.000000,4296.000000,1551.000000,NaN,5023.000000,1.000000,6.000000,212.000000,45.000000,572.000000,38.000000,1249.500000
Îles Salomon,0.000000,0.0,14.00000,29.000000,2.000000,7.000000,8.000000,3.00000,0.000000,8.000000,0.000000,49.000000,0.000000,2.000000,0.000000,8.133333
Moyenne,138.583333,619.0,3905.16092,1183.655172,296.523529,4303.816092,5067.333333,1536.91018,5493.097701,237.132948,480.948276,2733.655172,1611.114458,970.482759,921.687861,1966.606782


In [89]:
### 3 - Calculer le ratio entre la quantité destinés aux "Autres utilisations" et la disponibilité intérieure pour ces produits.
ratioAutresDI = analyseSousNutritionAutres.loc["Moyenne"]/analyseSousNutritionDI.loc["Moyenne"]*100
ratioAutresDI

Produit
Ananas                  0.309252
Bananes                 1.092392
Blé                     4.285183
Fruits, Autres          0.318058
Huile de Palme         70.224029
Lait - Excl Beurre      3.603951
Légumes, Autres         0.268605
Manioc                 20.185484
Maïs                   26.726423
Poissons Pelagiques     0.916537
Pommes                  0.545797
Riz (Eq Blanchi)        4.824519
Soja                    3.959992
Sucre Eq Brut          12.527175
Tomates                 0.835424
Global                  9.028571
Name: Moyenne, dtype: float64

In [90]:
### Calculer le ratio entre la quantité destinés à la nourriture animale et la quantité destinée à la nourriture (animale + humaine) :

## 4 - Calcul pour "Nourriture Animale"

# a) Restriction sur "Nourriture animale" pour analyseSousNutritionAnimaux :
analyseSousNutritionAnimaux = analyseSousNutrition.loc[analyseSousNutrition["Élément"] == "Aliments pour animaux"]

# b) Suppression de la colonne Céréales :
analyseSousNutritionAnimaux = analyseSousNutritionAnimaux.drop(columns = ["Céréales"])

# c) Restriction sur les 15 produits les plus exportés par les pays comptant des personnes en sous-nutrition :
analyseSousNutritionAnimaux["Parmi les 15 produits les plus exportés"] = analyseSousNutritionAnimaux["Produit"].isin(produitsLesPlusExportes)
analyseSousNutritionAnimaux = analyseSousNutritionAnimaux[analyseSousNutritionAnimaux["Parmi les 15 produits les plus exportés"] == True]

# d) Transformation en tableau Pivot
analyseSousNutritionAnimaux2 = analyseSousNutritionAnimaux.pivot_table("Valeur","Zone","Produit")
analyseSousNutritionAnimaux2.loc["Moyenne"] = analyseSousNutritionAnimaux2.mean()
analyseSousNutritionAnimaux2["Global"] = analyseSousNutritionAnimaux2.mean(axis=1)
analyseSousNutritionAnimaux2.tail()

Produit,Bananes,Blé,"Fruits, Autres",Huile de Palme,Lait - Excl Beurre,"Légumes, Autres",Manioc,Maïs,Poissons Pelagiques,Pommes,Riz (Eq Blanchi),Soja,Sucre Eq Brut,Tomates,Global
Zone,,,,,,,,,,,,,,,
Équateur,NaN,1.000000,NaN,NaN,124.000000,17.000000,0.00,830.000000,41.0,NaN,NaN,NaN,NaN,NaN,168.833333
États-Unis d'Amérique,NaN,6196.000000,NaN,NaN,481.000000,193.000000,288.00,128024.000000,208.0,NaN,NaN,2240.000000,NaN,NaN,19661.428571
Éthiopie,NaN,NaN,NaN,NaN,18.000000,NaN,NaN,667.000000,0.0,NaN,NaN,NaN,NaN,NaN,228.333333
Îles Salomon,NaN,NaN,NaN,NaN,0.000000,NaN,0.00,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,0.000000
Moyenne,45.842105,1309.777778,5.545455,0.0,780.695238,587.690476,870.56,3350.404908,121.4,33.15,414.740741,281.870968,3.913043,623.333333,602.066003


In [91]:
### Calculer le ratio entre la quantité destinés à la nourriture animale et la quantité destinée à la nourriture (animale + humaine) :

## 5 - Calcul pour "Nourriture Animale + humaine"

# a) Création d'un masque "Nourriture humaine" pour analyseSousNutritionAnimaux :
analyseSousNutritionNourritureH = analyseSousNutrition.loc[analyseSousNutrition["Élément"] == "Nourriture"]

# b) Suppression de la colonne Céréales
analyseSousNutritionNourritureH = analyseSousNutritionNourritureH.drop(columns = ["Céréales"])

# c) Création d'un masque pour les 15 produits les plus exportés par les pays comptant des personnes en sous-nutrition :
analyseSousNutritionNourritureH["Parmi les 15 produits les plus exportés"] = analyseSousNutritionNourritureH["Produit"].isin(produitsLesPlusExportes)
analyseSousNutritionNourritureH = analyseSousNutritionNourritureH[analyseSousNutritionNourritureH["Parmi les 15 produits les plus exportés"] == True]

# d) Jointure nourriture humaine + nourriture animale :
analyseSousNutritionNourritureH = analyseSousNutritionNourritureH.drop(columns = ["Élément"])
analyseSousNutritionNourritureH = analyseSousNutritionNourritureH.rename(columns={"Valeur":"Nourriture Humaine"})
analyseSousNutritionAnimaux = analyseSousNutritionAnimaux.rename(columns={"Valeur":"Nourriture Animale"})
analyseSousNutritionNourritureHA = pd.merge(analyseSousNutritionNourritureH, analyseSousNutritionAnimaux, how="left")
analyseSousNutritionNourritureHA["Nourriture"] = analyseSousNutritionNourritureHA["Nourriture Humaine"] + analyseSousNutritionNourritureHA["Nourriture Animale"] 
analyseSousNutritionNourritureHA

# d) Transformation en tableau Pivot
analyseSousNutritionNourritureHA = analyseSousNutritionNourritureHA.pivot_table("Nourriture","Zone","Produit")
analyseSousNutritionNourritureHA.loc["Moyenne"] = analyseSousNutritionNourritureHA.mean()
analyseSousNutritionNourritureHA["Global"] = analyseSousNutritionNourritureHA.mean(axis=1)
analyseSousNutritionNourritureHA.tail()

Produit,Bananes,Blé,"Fruits, Autres",Huile de Palme,Lait - Excl Beurre,"Légumes, Autres",Manioc,Maïs,Poissons Pelagiques,Pommes,Riz (Eq Blanchi),Soja,Sucre Eq Brut,Tomates,Global
Zone,,,,,,,,,,,,,,,
Équateur,NaN,618.00000,NaN,NaN,2321.000000,339.000000,40.000000,961.000000,90.000000,NaN,NaN,NaN,NaN,NaN,728.166667
États-Unis d'Amérique,NaN,31938.00000,NaN,NaN,81994.000000,21527.000000,334.000000,131941.000000,1132.000000,NaN,NaN,2252.000000,NaN,NaN,38731.142857
Éthiopie,NaN,NaN,NaN,NaN,4171.000000,NaN,NaN,4627.000000,1.000000,NaN,NaN,NaN,NaN,NaN,2933.000000
Îles Salomon,NaN,NaN,NaN,NaN,6.000000,NaN,3.000000,NaN,NaN,NaN,36.00000,NaN,NaN,NaN,15.000000
Moyenne,360.0,5628.69697,357.0,89.0,6361.352381,7661.190476,2610.880597,4187.591195,258.858065,227.1,4561.82716,439.491525,844.956522,3786.666667,2669.615111


In [92]:
### 6 - Calculer le ratio entre la quantité destinés à la nourriture animale et la quantité destinée à la nourriture (animale + humaine) :
ratioAnimauxNourriture = analyseSousNutritionAnimaux2.loc["Moyenne"]/analyseSousNutritionNourritureHA.loc["Moyenne"]*100
ratioAnimauxNourriture

Produit
Bananes                12.733918
Blé                    23.269645
Fruits, Autres          1.553349
Huile de Palme          0.000000
Lait - Excl Beurre     12.272473
Légumes, Autres         7.671007
Manioc                 33.343539
Maïs                   80.007927
Poissons Pelagiques    46.898288
Pommes                 14.597094
Riz (Eq Blanchi)        9.091549
Soja                   64.135700
Sucre Eq Brut           0.463106
Tomates                16.461268
Global                 22.552540
Name: Moyenne, dtype: float64

In [93]:
# Les trois produits, parmi les plus exportés, qui sont le plus destinés à d'autres utilisations que l'alimentaire sont :
ratioAutresDI.sort_values(ascending=False).head(3)

Produit
Huile de Palme    70.224029
Maïs              26.726423
Manioc            20.185484
Name: Moyenne, dtype: float64

In [94]:
# Les trois produits, parmi les plus exportés, qui sont le plus destinés à l'alimentation animale sont :
ratioAnimauxNourriture.sort_values(ascending=False).head(3)

Produit
Maïs                   80.007927
Soja                   64.135700
Poissons Pelagiques    46.898288
Name: Moyenne, dtype: float64

In [95]:
# QUESTION 13 ------------------------------------------------------------------------------------------------------------------
# Combien de tonnes de céréales pourraient être libérées si les USA diminuaient leur production de produits animaux de 10% ?

In [96]:
# Création d'un dataframe pour les USA :
analyseSousNutritionUSA  = analyseSousNutrition.loc[analyseSousNutrition["Zone"] == "États-Unis d'Amérique"]
analyseSousNutritionUSACereales = analyseSousNutritionUSA[analyseSousNutritionUSA["Céréales"] == True]

In [97]:
### Calcul du nombre de tonnes de céréales qui pouvant être libérés avec une baisse de la production de produits animaux de 10% :

# 1 - Calcul de la totalité de céréales consacrés à l'alimentation animale aux USA :
analyseSousNutritionUSACerealesAnimaux = analyseSousNutritionUSACereales.loc[analyseSousNutritionUSACereales["Élément"] == "Aliments pour animaux"]
totalUSACerealesAnimaux = analyseSousNutritionUSACerealesAnimaux["Valeur"].sum(axis=0)

# 2 - Calcul du nombre de tonnes de céréales libérés par une baisse de 10% de la production de produits animaux.
cerealesLiberees = totalUSACerealesAnimaux * 0.1 * 1000
cerealesLiberees

14009600.0

In [98]:
# QUESTION 14 ------------------------------------------------------------------------------------------------------------------
# En Thaïlande, quelle proportion de manioc est exportée ? Quelle est la proportion de personnes en sous nutrition ?

In [99]:
# Création d'un dataframe pour la Thaïlande :
analyseSousNutritionThaï = analyseSousNutrition.loc[analyseSousNutrition["Zone"] == "Thaïlande"]
analyseSousNutritionThaï.head()

,Zone,Élément,Produit,Valeur,Population,Nbre de personnes en sous-nutrition,% de la population en sous-alimentation,Céréales
33742,Thaïlande,Production,Viande de Bovins,195.0,67011000,5600000.0,8.356837,False
33743,Thaïlande,Importations - Quantité,Viande de Bovins,63.0,67011000,5600000.0,8.356837,False
33744,Thaïlande,Exportations - Quantité,Viande de Bovins,106.0,67011000,5600000.0,8.356837,False
33745,Thaïlande,Disponibilité intérieure,Viande de Bovins,152.0,67011000,5600000.0,8.356837,False
33746,Thaïlande,Traitement,Viande de Bovins,-0.0,67011000,5600000.0,8.356837,False


In [100]:
# 1 - Calcul de la proportion de manioc exporté par la Thaïlande :

# Restriction sur le manioc :
analyseSousNutritionThaïManioc = analyseSousNutritionThaï.loc[analyseSousNutritionThaï["Produit"] == "Manioc"]

# Transformation en tableau pivot :
analyseSousNutritionThaïManioc = analyseSousNutritionThaïManioc.pivot_table("Valeur", "Élément")

# Calcul de la proportion de manioc exporté par la Thaïlande :
PropManiocExport = analyseSousNutritionThaïManioc.loc["Exportations - Quantité"]/analyseSousNutritionThaïManioc.loc["Production"]*100
PropManiocExport

Valeur    83.41273
dtype: float64

In [101]:
# 2 - Calcul de la proportion de personnes en sous-nutrition en Thaïlande ? (Proportion déjà calculée en amont - entrée 79)
fr_sousalimentation3.loc[fr_sousalimentation3["Zone"] == "Thaïlande"]["% de la population en sous-alimentation"]

925    8.356837
Name: % de la population en sous-alimentation, dtype: float64

In [102]:
# QUESTION 15 -----------------------------------------------------------------------------------------------------------------
# Créer une table "population" contenant la population de chaque pays en 2013.
# Proposition de clé primaire : [code_pays]

In [103]:
# Projection :
population = fr_population[["Zone","Code zone","Année","Population"]]

# Dénomination des colonnes selon les consignes :
population = population.rename(columns={"Code zone":"code_pays", "Zone":"pays", "Année":"annee", "Population":"population"})

# Export vers csv :
# population.to_csv("population.csv", index = False, encoding="ISO-8859-1")
population.head()

,pays,code_pays,annee,population
0,Afghanistan,2,2013,30552000
1,Afrique du Sud,202,2013,52776000
2,Albanie,3,2013,3173000
3,Algérie,4,2013,39208000
4,Allemagne,79,2013,82727000


In [104]:
# QUESTION 16 -----------------------------------------------------------------------------------------------------------------
# Créer une table "dispo_alim" contenant la population de chaque pays en 2013 :
# Proposition de clé primaire : artificielle ou [code_pays,code_produit].

In [105]:
# Création du dataframe dispo_alim sur le modèle fr_alimentation
dispo_alim = fr_alimentation

# Création de la colonne "origin" :
fr_vegetaux["Code Produit"].unique()
produitsVeg = [2511, 2805, 2513, 2514, 2517, 2520, 2531, 2536, 2537, 2542, 2543,
       2745, 2549, 2551, 2557, 2559, 2560, 2561, 2563, 2570, 2571, 2572,
       2573, 2574, 2575, 2577, 2579, 2580, 2586, 2601, 2605, 2611, 2614,
       2615, 2617, 2618, 2619, 2620, 2625, 2630, 2633, 2635, 2640, 2645,
       2655, 2656, 2658, 2680, 2899, 2515, 2516, 2518, 2532, 2533, 2535,
       2534, 2546, 2547, 2555, 2556, 2558, 2562, 2576, 2578, 2582, 2602,
       2612, 2613, 2616, 2641, 2642, 2657, 2659, 2541, 2581]
dispo_alim["origin"] = dispo_alim["Code Produit"].isin(produitsVeg).replace({True:"vegetal", False:"animal"})

dispo_alim = dispo_alim[["Élément","Zone","Code zone","Année","Produit","Code Produit","origin","Valeur"]]

# Restrictions :
dispo_alim1 = dispo_alim.loc[dispo_alim["Élément"]=="Nourriture"].drop(columns=["Élément"]).rename(columns={"Valeur":"dispo_alim_tonnes"})
dispo_alim2 = dispo_alim.loc[dispo_alim["Élément"]=="Disponibilité alimentaire (Kcal/personne/jour)"].drop(columns=["Élément"]).rename(columns={"Valeur":"dispo_alim_kcal_p_j"})
dispo_alim3 = dispo_alim.loc[dispo_alim["Élément"]=="Disponibilité de protéines en quantité (g/personne/jour)"].drop(columns=["Élément"]).rename(columns={"Valeur":"dispo_prot"})
dispo_alim4 = dispo_alim.loc[dispo_alim["Élément"]=="Disponibilité de matière grasse en quantité (g/personne/jour)"].drop(columns=["Élément"]).rename(columns={"Valeur":"dispo_mat_gr"})

# Conversion des milliers de tonnes
dispo_alim1["dispo_alim_tonnes"] = dispo_alim1["dispo_alim_tonnes"] * 1000

# Jointure des différentes restrictions :
dispo_alim = pd.merge(dispo_alim2, dispo_alim1, how="left")
dispo_alim = pd.merge(dispo_alim, dispo_alim3, how="left")
dispo_alim = pd.merge(dispo_alim, dispo_alim4, how="left")

# Dénomination des colonnes selon les consignes :
dispo_alim = dispo_alim.rename(columns={"Code zone":"code_pays", "Zone":"pays", "Année":"annee", "Produit":"produit", "Code Produit":"code_produit"})

# Export vers csv :
# dispo_alim.to_csv("dispo_alim.csv", index = False, encoding="ISO-8859-1")
dispo_alim.head()

,pays,code_pays,annee,produit,code_produit,origin,dispo_alim_kcal_p_j,dispo_alim_tonnes,dispo_prot,dispo_mat_gr
0,Afghanistan,2,2013,Viande de Bovins,2731,animal,27.0,140000.0,1.89,2.10
1,Afghanistan,2,2013,Viande d'Ovins/Caprins,2732,animal,31.0,150000.0,1.91,2.55
2,Afghanistan,2,2013,Viande de Volailles,2734,animal,9.0,75000.0,0.87,0.53
3,Afghanistan,2,2013,"Viande, Autre",2735,animal,1.0,12000.0,0.17,0.06
4,Afghanistan,2,2013,Abats Comestible,2736,animal,5.0,53000.0,0.77,0.20


In [106]:
# QUESTION 17 -----------------------------------------------------------------------------------------------------------------
# Créer une table "dispo_alim" contenant la population de chaque pays en 2013 :
# Proposition de clé primaire : clé artificielle ou [code_pays,code_produit].

In [107]:
# Création du dataframe equilibre_prod sur le modèle fr_alimentation
equilibre_prod = fr_alimentation
equilibre_prod = equilibre_prod[["Zone","Code zone","Année","Élément","Produit","Code Produit","Valeur"]]

# Restrictions :
equilibre_prod1 = equilibre_prod.loc[equilibre_prod["Élément"]=="Disponibilité intérieure"].drop(columns=["Élément"]).rename(columns={"Valeur":"dispo_int"})
equilibre_prod2 = equilibre_prod.loc[equilibre_prod["Élément"]=="Aliments pour animaux"].drop(columns=["Élément"]).rename(columns={"Valeur":"alim_ani"})
equilibre_prod3 = equilibre_prod.loc[equilibre_prod["Élément"]=="Semences"].drop(columns=["Élément"]).rename(columns={"Valeur":"semences"})
equilibre_prod4 = equilibre_prod.loc[equilibre_prod["Élément"]=="Pertes"].drop(columns=["Élément"]).rename(columns={"Valeur":"pertes"})
equilibre_prod5 = equilibre_prod.loc[equilibre_prod["Élément"]=="Traitement"].drop(columns=["Élément"]).rename(columns={"Valeur":"transfo"})
equilibre_prod6 = dispo_alim1.drop(columns=["origin"]).rename(columns={"dispo_alim_tonnes":"nourriture"})
equilibre_prod7 = equilibre_prod.loc[equilibre_prod["Élément"]=="Autres utilisations (non alimentaire)"].drop(columns=["Élément"]).rename(columns={"Valeur":"autres_utilisations"})

# Jointure des différentes restrictions :
equilibre_prod = pd.merge(equilibre_prod1, equilibre_prod2, how="left")
equilibre_prod = pd.merge(equilibre_prod, equilibre_prod3, how="left")
equilibre_prod = pd.merge(equilibre_prod, equilibre_prod4, how="left")
equilibre_prod = pd.merge(equilibre_prod, equilibre_prod5, how="left")
equilibre_prod = pd.merge(equilibre_prod, equilibre_prod6, how="left")
equilibre_prod = pd.merge(equilibre_prod, equilibre_prod7, how="left")

# Conversion des tonnes en milliers de tonnes
equilibre_prod["nourriture"] = equilibre_prod["nourriture"] / 1000

# Dénomination des colonnes selon les consignes :
equilibre_prod = equilibre_prod.rename(columns={"Code zone":"code_pays", "Zone":"pays", "Année":"annee", "Produit":"produit", "Code Produit":"code_produit"})

# Export vers csv :
# equilibre_prod.to_csv("equilibre_prod.csv", index = False, encoding="ISO-8859-1")
equilibre_prod.head()

,pays,code_pays,annee,produit,code_produit,dispo_int,alim_ani,semences,pertes,transfo,nourriture,autres_utilisations
0,Afghanistan,2,2013,Viande de Bovins,2731,140.0,NaN,NaN,NaN,NaN,140.0,NaN
1,Afghanistan,2,2013,Viande d'Ovins/Caprins,2732,150.0,NaN,NaN,NaN,NaN,150.0,NaN
2,Afghanistan,2,2013,Viande de Volailles,2734,75.0,NaN,NaN,NaN,NaN,75.0,NaN
3,Afghanistan,2,2013,"Viande, Autre",2735,12.0,NaN,NaN,NaN,NaN,12.0,NaN
4,Afghanistan,2,2013,Abats Comestible,2736,53.0,NaN,NaN,NaN,NaN,53.0,NaN


In [108]:
# QUESTION 18 -----------------------------------------------------------------------------------------------------------------
# Créer une table "sous_nutrition" contenant le nombre de personnes en sous-nutrition pour chaque pays en 2013 :
# Proposition de clé primaire : clé artificielle ou [code_pays,année].

In [109]:
# Restriction :
sous_nutrition = fr_sousalimentation[["Zone","Code zone","Année","Valeur"]]

# Dénomination des colonnes selon les consignes :
sous_nutrition = sous_nutrition.rename(columns={"Zone":"pays","Code zone":"code_pays","Année":"annee", "Valeur":"nb_personnes"})

# Export vers csv :
# sous_nutrition.to_csv("sous_nutrition.csv", index = False, encoding="ISO-8859-1")
sous_nutrition.head()

,pays,code_pays,annee,nb_personnes
0,Afghanistan,2,2012-2014,7900000.0
1,Afghanistan,2,2013-2015,8800000.0
2,Afghanistan,2,2014-2016,9600000.0
3,Afghanistan,2,2015-2017,10200000.0
4,Afghanistan,2,2016-2018,10600000.0


In [110]:
# QUESTION 19 -----------------------------------------------------------------------------------------------------------------
# Écrivez les requêtes SQL permettant de connaître :

In [111]:
#1 - Les 10 pays ayant le plus haut ratio disponibilité alimentaire/habitant en termes de protéines (en kg) par habitant,
    # SELECT 
    # pays,
    # sum(dispo_prot)/1000*365 as kilo_prot
    # FROM dispo_alim
    # GROUP BY pays
    # ORDER BY kilo_prot DESC
    # LIMIT 10 ;

#2 - puis en termes de kcal par habitant :
    # SELECT 
    # pays,
    # sum(dispo_alim_kcal_p_j)*365 as kcal_p
    # FROM dispo_alim
    # GROUP BY pays
    # ORDER BY kcal_p DESC
    # LIMIT 10 ;

In [112]:
#3 - Pour l'année 2013, les 10 pays ayant le plus faible ratio disponibilité alimentaire/habitant en termes de protéines :
    # SELECT 
    # pays,
    # sum(dispo_prot)/1000*365 as kilo_prot
    # FROM dispo_alim
    # GROUP BY pays
    # ORDER BY kilo_prot ASC
    # LIMIT 10 ;

In [113]:
#4 - La quantité totale (en kg) de produits perdus par pays en 2013 :
    # SELECT 
    # pays,
    # sum(pertes)*1000000 as pertesKg
    # FROM equilibre_prod
    # GROUP BY pays
    # ORDER BY pertesKg DESC ;

In [114]:
#5 - Les 10 pays pour lesquels la proportion de personnes sous-alimentées est la plus forte :
    # SELECT 
    # pays,
    # sous_nutrition.annee,
    # nb_personnes,
    # population,
    # nb_personnes/population*100 AS ratio_sous_alim
    # FROM 
    # sous_nutrition, 
    # population 
    # WHERE
    # population.pays = sous_nutrition.pays
    # AND sous_nutrition.annee="2012-2014" 
    # ORDER BY
    # ratio_sous_alim DESC
    # LIMIT 10 ;

In [115]:
#6 - Les 10 produits pour lesquels le ratio Autres utilisations/Disponibilité intérieure est le plus élevé :
    # SELECT 
    # produit,
    # sum(autres_utilisations)/sum(dispo_int)*100 as ratio
    # FROM equilibre_prod
    # GROUP BY produit
    # ORDER BY ratio DESC
    # LIMIT 10 ;

In [116]:
# QUESTION 20 -----------------------------------------------------------------------------------------------------------------
# pour quelques uns des produits identifiés dans cette dernière requête SQL, 
# supposez quelles sont ces "autres utilisations" possibles :

In [117]:
# - Savons et detergeants
# - Cosmétiques
# - Biocarburants et biolubrifiants
# - Plastiques
# - Matériaux
# - Industrie textile
# - Chimie
# - Pharmaceutique

In [118]:
# ENRICHISSEZ VOTRE ANALYSE ----------------------------------------------------------------------------------------------------
# Combien de personnes décèdent des causes de la faim ?
# Quelles sont les prévisions de population mondiale en 2050 ?

In [119]:
# 1 - 25 000 personnes dont plus de 10 000 enfants meurent chaque jour de la faim et de ses 
    # conséquences.Cela fait 9 125 000 personnes par an.
    # Plus de 2 milliards d'êtres humains souffrent de carences en oligo-éléments, en vitamines,
    # en minéraux.("la faim cachée")
    # Grande importance des conflits dans la constitution de ces problèmes.

In [120]:
# 2 - la population humaine devrait approcher des 10 milliards d'êtres en 2050 d'après un rapport de l'ONU.
# Selon ce dernier, il faudra augmenter la production, ce qui n'est pas insurmontable mais pose d'
# importantes externalités :
    # Impact énorme sur l'environnement et le réchauffement planétaire.
    # Problème de main d'oeuvre alors que la population aura beaucoup vieilli.